# lit recomender system

In [ ]:
# need to run in `nlp` conda environment

In [1]:
cd /home/jeremy/workshop/nlp-rec

/home/jeremy/workshop/nlp-rec


In [2]:
import pandas as pd
import time
from pathlib import Path
import numpy as np
import requests
from lxml import etree

## seed arxiv links

In [283]:
z = pd.read_csv('data/arxiv.csv')

In [42]:
def id_from_url(url):
    "Parse arxiv paper id from url."
    parts = url.split('/')
    if 'arxiv.org' in parts: 
        i = parts.index('abs')
        return parts[i+1]
    elif 'www.arxiv-vanity.com' in parts:
        i = parts.index('papers')
        return parts[i+1]
    else:
        # FIXME: printing failures to recognize for now.
        print(url)

In [43]:
z.Url[0]

'http://arxiv.org/abs/1606.05830'

In [287]:
ids = list(map(id_from_url, pd.read_csv('data/arxiv.csv').Url))
ids

['1606.05830',
 '1703.00848',
 '1406.2661',
 '1606.05908',
 '1611.08097',
 '1801.05894',
 '1404.7828',
 '1605.09782',
 '1503.07810',
 '1803.03835',
 '1606.04080',
 '1606.02819',
 '1709.01846',
 '1801.05401',
 '1802.09210',
 '1802.03646',
 '1806.03723',
 '1710.11431',
 '1801.06637',
 '1711.10561',
 '1711.10566',
 '1711.00489',
 '1609.03540',
 '1707.06315',
 '1804.08646',
 '1010.2952',
 '1502.06229',
 '1809.01604',
 '1801.00826',
 '1801.00718',
 '1101.1438',
 '1610.09887',
 '1710.09412',
 '1608.06993',
 '1512.03965',
 '1710.05941',
 '1702.03118',
 '1810.02880',
 '1708.00588',
 '1703.03400',
 '1710.09412',
 '1612.08544',
 '1705.09792',
 '1802.01542',
 '1605.05509',
 '1806.03723',
 '1609.03499',
 '1806.03723',
 '1712.10082',
 '1810.06603',
 '1610.01439',
 '1803.00894',
 '1609.06401',
 '1901.00246',
 '1606.03490',
 '1010.5890',
 '1604.01444',
 '1609.03499',
 '1703.04691',
 '1612.03144',
 '1603.07285',
 '1611.06473',
 '1703.02930',
 '1610.09887',
 '1711.08920',
 '1412.6830',
 '1904.12064',
 

In [111]:
id_list = ','.join(ids)

In [ ]:
# add papers from the zotero lib to pos

#ids = list(map(id_from_url, pd.read_csv('data/arxiv.csv').Url))

#for id in ids:
    #p = list(Path('papers/').glob(f"{id}*.xml"))
    #id = id_from_path(p[0])
    #save_label(id, "1")

## arxiv downloader

In [4]:
def paper(id_list):
    "Retrive metadata xml for paper."
    url = f"http://export.arxiv.org/api/query?id_list={id_list};max_results=500"
    r = requests.get(url)
    assert r.status_code == 200
    return r.content


def cache_paper(id):
    "Cache metadata xml for paper."
    pth = f"papers/{id}.xml"
    xml = paper(id)
    with open(pth, 'w') as f:
        f.write(xml)

In [5]:
def entries(content):
    tree = etree.fromstring(content)
    return tree.findall("{http://www.w3.org/2005/Atom}entry")

In [6]:
def entry_id(entry):
    return id_from_url(entry.find('{http://www.w3.org/2005/Atom}id').text)

In [7]:
def cache_entry(entry):
    id = entry_id(entry)
    with open(f'papers/{id}.xml', 'wb') as fout:
        fout.write(etree.tostring(entry))

def cache_entries(content):
    cnt = 0
    for entry in entries(content):
        cache_entry(entry)
        cnt += 1
    return cnt
    

## download sample of arxiv meta data

In [8]:
def search(search_term, start=0, max_results=50):
    search_term = '+'.join(search_term.split())  # FIXME: do proper url encoding
    url = f"http://export.arxiv.org/api/query?search_query={search_term}&start={start}&max_results={max_results}"
    r = requests.get(url)
    return r
    assert r.status_code == 200
    return r.content

In [9]:
def cache_search(search_term, limit=3000, chunk_size=500, delay=5):
    cnt = 0
    for s in range(0, limit, chunk_size):
        r = search(search_term, max_results=chunk_size, start=s)
        print(s, r.status_code, len(list(entries(r.content))))
        for p in entries(r.content):
            #print('x', entry_id(p))
            cache_entry(p)
            cnt += 1
        time.sleep(delay)
    return cnt

seed with results from following search terms
- machine learning
- deep learning
- gas turbine
- predictive maintenence
- interpolation
- lattice regression
- lattice networks
- spline
- anomaly detection
- condition based monitoring
- wind turbine
- power plant
- flooding
- streamflow
- hydrological
- weather
- neural network

In [10]:
# uncomment to run search
# I used the web interface to run the query and get a reasonable bound for the limit.

#cache_search('neural network', limit=30001, chunk_size=500, delay=2)

## show paper and manually label

In [11]:
def xml(id):
    with open(f"papers/{id}.xml") as f:
        data = f.read()
    return data


def save_label(id, label):
    with open(f"labels/{label}/{id}", "w") as f:
        f.write(label)

        
def manually_label(id):
    print(xml(id))
    label = input("Interested (0|1)? ")
    if label == 'q':
        return
    save_label(id, label)
    return label
    

def id_from_path(pth):
    pth = str(pth).split('/')[-1]
    if pth.endswith('.xml'):
        return pth[:-4]
    else:
        return pth

In [114]:
# papers retrieved
ids = Path('papers/').glob('*.xml')
ids = set(map(id_from_path, ids))
len(ids)

# papers already labeled
pos = set(map(id_from_path, Path('labels/1').glob('*')))
neg = set(map(id_from_path, Path('labels/0').glob('*')))

to_label = list(ids - pos - neg)

print(len(pos), len(neg), len(to_label))


197 312 50259


In [115]:
197 + 312

509

In [113]:
# set QTY and run to interactivly label a random sample of unlabled articles
QTY = 50

l = np.random.choice(to_label, QTY, replace=False)
for id in l:
    manually_label(id)

<entry xmlns="http://www.w3.org/2005/Atom">
    <id>http://arxiv.org/abs/1908.01268v1</id>
    <updated>2019-08-04T03:59:24Z</updated>
    <published>2019-08-04T03:59:24Z</published>
    <title>Identify the stiffness of DNA via deep learning</title>
    <summary>  DNA detection is of great significance in the point-of-care diagnostics. The
stiffness of DNA, varying with its sequence and mechanochemical environment,
could be a potential marker for DNA identification. The steric configurations
of DNA fragments with different stiffness were simulated by employing the
Kirchhoff theory of thin elastic rods. We identified the stiffness of DNA with
the trained convolutional neural network on the simulated image set. The
identification accuracy reached 99.85%. The stiffness-based identification
provided a promising approach for DNA detection.
</summary>
    <author>
      <name>Haiqian Yang</name>
    </author>
    <author>
      <name>Liu Yang</name>
    </author>
    <author>
      <name>Sha

Interested (0|1)?  1


<entry xmlns="http://www.w3.org/2005/Atom">
    <id>http://arxiv.org/abs/1708.00684v1</id>
    <updated>2017-08-02T10:20:22Z</updated>
    <published>2017-08-02T10:20:22Z</published>
    <title>OmniArt: Multi-task Deep Learning for Artistic Data Analysis</title>
    <summary>  Vast amounts of artistic data is scattered on-line from both museums and art
applications. Collecting, processing and studying it with respect to all
accompanying attributes is an expensive process. With a motivation to speed up
and improve the quality of categorical analysis in the artistic domain, in this
paper we propose an efficient and accurate method for multi-task learning with
a shared representation applied in the artistic domain. We continue to show how
different multi-task configurations of our method behave on artistic data and
outperform handcrafted feature approaches as well as convolutional neural
networks. In addition to the method and analysis, we propose a challenge like
nature to the new aggreg

Interested (0|1)?  0


<entry xmlns="http://www.w3.org/2005/Atom">
    <id>http://arxiv.org/abs/1506.01660v2</id>
    <updated>2016-03-09T16:26:26Z</updated>
    <published>2015-06-04T17:27:56Z</published>
    <title>Transition from lognormal to chi-square superstatistics for financial
  time series</title>
    <summary>  Share price returns on different time scales can be well modelled by a
superstatistical dynamics. Here we provide an investigation which type of
superstatistics is most suitable to properly describe share price dynamics on
various time scales. It is shown that while chi-square superstatistics works
well on a time scale of days, on a much smaller time scale of minutes the price
changes are better described by lognormal superstatistics. The system dynamics
thus exhibits a transition from lognormal to chi-square superstatistics as a
function of time scale. We discuss a more general model interpolating between
both statistics which fits the observed data very well. We also present results
on co

...

Interested (0|1)?  0


<entry xmlns="http://www.w3.org/2005/Atom">
    <id>http://arxiv.org/abs/1206.6444v1</id>
    <updated>2012-06-27T19:59:59Z</updated>
    <published>2012-06-27T19:59:59Z</published>
    <title>Statistical Linear Estimation with Penalized Estimators: an Application
  to Reinforcement Learning</title>
    <summary>  Motivated by value function estimation in reinforcement learning, we study
statistical linear inverse problems, i.e., problems where the coefficients of a
linear system to be solved are observed in noise. We consider penalized
estimators, where performance is evaluated using a matrix-weighted two-norm of
the defect of the estimator measured with respect to the true, unknown
coefficients. Two objective functions are considered depending whether the
error of the defect measured with respect to the noisy coefficients is squared
or unsquared. We propose simple, yet novel and theoretically well-founded
data-dependent choices for the regularization parameters for both cases that
av

Interested (0|1)?  1


<entry xmlns="http://www.w3.org/2005/Atom">
    <id>http://arxiv.org/abs/1611.09934v2</id>
    <updated>2017-08-22T09:19:11Z</updated>
    <published>2016-11-29T23:06:21Z</published>
    <title>Neural Network Models for Software Development Effort Estimation: A
  Comparative Study</title>
    <summary>  Software development effort estimation (SDEE) is one of the main tasks in
software project management. It is crucial for a project manager to efficiently
predict the effort or cost of a software project in a bidding process, since
overestimation will lead to bidding loss and underestimation will cause the
company to lose money. Several SDEE models exist; machine learning models,
especially neural network models, are among the most prominent in the field. In
this study, four different neural network models: Multilayer Perceptron,
General Regression Neural Network, Radial Basis Function Neural Network, and
Cascade Correlation Neural Network are compared with each other based on: (1)
predi

Interested (0|1)?  0


## make a dataset of all unlabled data

In [116]:
# papers retrieved
ids = Path('papers/').glob('*.xml')
ids = list(map(id_from_path, ids))


In [117]:
# want id, title, abstract in a dataframe

def normalize(id):
    with open(f'papers/{id}.xml') as f:
        xml = f.read()
    ns = {"atom":"http://www.w3.org/2005/Atom"}
    t = etree.fromstring(xml)
    title = ' '.join(t.find('atom:title', ns).text.split())
    abstract = ' '.join(t.find('atom:summary', ns).text.split())
    return id, title, abstract

In [118]:
res = list(map(normalize, ids))
df = pd.DataFrame.from_records(res, columns=['id', 'title', 'abstract'])
df.to_csv('data/unlabeled.tsv', sep='\t', index=False)

## make a dataset of labels

In [122]:
pos = list(set(map(id_from_path, Path('labels/1').glob('*'))))
neg = list(set(map(id_from_path, Path('labels/0').glob('*'))))

In [123]:
p = pd.DataFrame({"id": pos, "label":1})
n = pd.DataFrame({"id": neg, "label":0})

In [124]:
labeled = pd.concat([p, n])

In [125]:
labeled.to_csv('data/labels.csv', index=False)

## merge title and abstract into labeled data

In [126]:
df = df.set_index('id')

In [127]:
labeled = labeled.set_index('id')

In [128]:
labeled = labeled.join(df).reset_index()

In [129]:
labeled.to_csv('data/labeled.tsv', sep='\t', index=False)